# Integración numérica de la ecuación de onda
Utilizaré distintos métodos (pues el profe hinteó que los algorimos usuales no van a funcionar, pero quisiera ver qué es lo que le pasa) para integrar numéricamente la ecuación de onda adimensionalizada:
$$
\partial_{tt} u(t, x) = \partial_{xx} u(t, x)
$$


## Algoritmo naive:

La idea es usar la clásica discretización de la segunda derivada a ambos lados:

$$
\partial_{zz} f(z) \approx \frac{f(z + h) - 2 f(z) + f(z - h)}{h^2} 
$$

Y con esto, despejar el valor de $u(t + \Delta t, x)$ en función de valores de tiempos anteriores y otras posiciones. Para el primer paso de tiempo simplemente usaré la condición inicial, es decir, supongo velocidad inicial 0 para cada $u(\cdot, x)$. También usaré condiciones de borde fijas.


In [54]:
using LinearAlgebra
using GLMakie
using Printf
using Integrals

const x_largo = 1.0
const N_x = 101
const Δx = x_largo/(N_x - 1)
const Δt = Δx*1.0
const s = (Δt/Δx)^2
const sMatrix = Tridiagonal(ones(N_x-3) .* s, ones(N_x-2) .* 2 .* (1 - s), ones(N_x-3) .* s)
const bordes = 0.0
const x = range(0, x_largo, length=N_x)

println("s = ", s)

s = 1.0


In [55]:
function paso_temporal!(u, u_anterior)
    buffer = copy(u)
    u[2:end-1] .= sMatrix * u[2:end-1] .- u_anterior[2:end-1]
    u[1] = bordes
    u[end] = bordes # En verdad esto no es necesario.
    u[2] += s * bordes 
    u[end-1] += s * bordes
    u_anterior .= buffer
end

function actualizar_animacion!(i)
    lines[1][] = Point2f[[x[i], u[i]] for i in 1:N_x] # Horrible sintaxis.
    text.text[] = @sprintf("Energía = %.3f", energía(u, u_anterior))
    paso_temporal!(u, u_anterior)
    return
end

function energía(u, u_anterior)
    dudt = (u - u_anterior)/Δt 
    dudx = (u[2:end] - u[1:end-1])/Δx # Aproximaciones esooo 
    pot_energy_problem = SampledIntegralProblem(dudt.^2, x)  # En esta expresión uso densidad de masa unitaria.
    kin_energy_problem = SampledIntegralProblem(dudx.^2, x[1:end-1])
    method = TrapezoidalRule()
    pot = solve(pot_energy_problem, method)
    kin = solve(kin_energy_problem, method)
    return (pot.u + kin.u)/2.0
end


energía (generic function with 1 method)

In [56]:
# Animación:

u = sin.(2π .* x)
u_anterior = copy(u)
fig = Figure()
ax = Axis(fig[1, 1], limits=(0, 1, -1, 1))
text = text!(ax, 0.1, 0.9, text=@sprintf("Energía = %.3f", energía(u, u_anterior)))
lines = lines!(ax, x, u)
nframes = 1080
framerate = 60

record(actualizar_animacion!, fig, "modo_normal.mp4", 1:nframes; framerate=framerate)


"modo_normal.mp4"

Ahora otra condición inicial

In [57]:
function u_func(x)
    if x < 1/3 || x > 2/3
        return 0.0
    elseif x < 1/2
        return x - 1/3
    else
        return 2/3 - x
    end
end

using Printf

u = u_func.(x)
u_anterior = copy(u)
fig = Figure()
ax = Axis(fig[1, 1], limits=(0, 1, -1/3, 1/3))
lines = lines!(ax, x, u)
text = text!(ax, 0.0, 0.16, text=@sprintf("Energía = %.3f", energía(u, u_anterior)))
nframes = 1080
framerate = 60

record(actualizar_animacion!, fig, "triángulo.mp4", 1:nframes; framerate=framerate)


"triángulo.mp4"

Adjuntaré un análisis de estabilidad que hice, que me asegura que cuando $ s\leq 1$ el método es estable. También vemos que para $s<1$, la condición inicial no se rearma en un periodo. Esto sólo puede ocurrir si la relación de dispersión numérica es distinta a la exacta (que es $\omega = k$). Vemos que hay unas oscilaciones extrañas ahí metidas cuando $s = 1$, una especie de ruido (proveniente del modo más pequeño posible debido a la discretización) que me dificulta calcular la energía, dandome valores muy oscilantes. Sin embargo, está claro que la energía efectivamente se conserva pues al cabo de varias oscilaciones, la condición inicial se rearma.

## Corrección no lineal

Agregamos ahora la primera corrección no-lineal de la ecuación. La ecuación de onda no lineal es:

$$
\partial_{tt} u(t, x) = \partial_{xx} u(t, x) (1 + \frac{3}{2}\partial_x u(t, x)^2)
$$

Esto me agrega al RHS un término tipo:
$$
a \frac{u^2(t, x + h) - 2 u(t, x) u(t, x + h) + u^2(t, x)}{h^2}
$$

O bien, usando la derivada al otro lado:
$$
a \frac{u^2(t, x) - 2 u(t, x) u(t, x - h) + u^2(t, x-h)}{h^2}

tip leer Zeldovich Shock waves